# Visualizing ADCIRC data

[Datashader](http://datashader.org)'s support for irregular triangular meshes allows datasets from such simulations to be rendered onscreen efficiently. This notebook shows an example of rendering a ADCIRC tidal database of the US Eastern coast, Once Datashader and the dataset are installed, you can run this notebook yourself to get a live version with interactive zooming for the plots that support it.

Here you can enable the wheel-zoom tool in the toolbar, then use scrolling and your pointer to zoom and pan in the plot. As always with datashader, the data is provided to the browser in only one resolution to start with, and it will be updated when you zoom in

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
import datashader.utils as du
import pandas as pd
import numpy as np

In [ ]:
from gridgeo.ugrid import ugrid
import netCDF4

In [ ]:
url='http://gamone.whoi.edu/thredds/dodsC/usgs/vault0/models/tides/ec2015/f53.ncml'
nc = netCDF4.Dataset(url)
u = ugrid(nc)

z = nc['Amp'][0,:]
#z = nc['depth'][:]


In [ ]:
v = np.vstack((u['nodes']['x'], u['nodes']['y'], z)).T

In [ ]:
v

In [ ]:
verts = pd.DataFrame(v, columns=['x','y','z'])
tris = pd.DataFrame(u['faces'].astype('int'), columns=['v0','v1','v2'])

In [ ]:
%time mesh = du.mesh(verts, tris)

In [ ]:
from matplotlib.cm import jet
cvs = ds.Canvas(plot_height=500, plot_width=500)
%time agg = cvs.trimesh(verts, tris, mesh=mesh)

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade

hv.extension("bokeh")

In [ ]:
import geoviews as gv

%opts WMTS [width=500 height=500]

tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

%time points = gv.operation.project_points(gv.Points(verts, vdims=['z']))

tiles * datashade(hv.TriMesh((tris, points)), cmap=jet, 
                  normalization='linear', aggregator=ds.mean('z'), precompute=False)

Here you can enable the wheel-zoom tool in the toolbar, then use scrolling and your pointer to zoom and pan in the plot.  As always with datashader, the data is provided to the browser in only one resolution to start with, and it will be updated when you zoom in only if you have a running Python process, and are not just viewing this on a static web page.